## Iterative vs fragment-based mapping

__running time__: < 15 min

Iterative mapping first proposed by <a name="ref-1"/>[(Imakaev et al., 2012)](#cite-Imakaev2012a), allows to map usually a high number of reads. However other methodologies, less "brute-force" can be used to take into account the chimeric nature of the Hi-C reads.

A simple alternative is to allow split mapping.

Another way consists in _pre-truncating_ <a name="ref-1"/>[(Ay and Noble, 2015)](#cite-Ay2015a) reads that contain a ligation site and map only the longest part of the read <a name="ref-2"/>[(Wingett et al., 2015)](#cite-Wingett2015).

Finally, an intermediate approach, _fragment-based_, consists in mapping full length reads first, and than splitting unmapped reads at the ligation sites <a name="ref-1"/>[(Serra et al. 2017)](#cite-Serra2017).

![mapping strategies](images/mapping.png)

### Advantages of iterative mapping

- It's the only solution when no restriction enzyme has been used (i.e. micro-C)
- Can be faster when few windows (2 or 3) are used

### Advantages of fragment-based mapping 

- Generally faster
- Safer: mapped reads are generally larger than 25-30 nt (the largest window used in iterative mapping). Less reads are mapped, but the difference is usually canceled or reversed when looking for "valid-pairs".

## Mapping

`tadbit map` tool can be used to perform either iterative or fragment-based mapping, or a combination of both.

In the course we will use fragment-based mapping which is generally more efficient for a conventional Hi-C experiment. If we choose to use the iterative mapping strategy the `tadbit map` command will be exactly the same but adding the `--iterative` option: 

### Fragment-based mapping

The fragment-based mapping strategy works in 2 steps: 
1. The read-ends are mapped entirely, assuming that no ligation occurred in them. 
2. For unmapped read-ends, the function searches for a ligation site (e.g. in the case of MboI this would correspond to `GATCGATC` and in the case of HindIII to `AAGCTAGCTT`). The read-end is split accordingly replacing the ligation site by two RE sites:

```
                        read-end-part-one---AAGCTAGCTT----read-end-part-two
     will be split in:
                        read-end-part-one---AAGCTT
     and:
                                                AAGCTT----read-end-part-two
```

_Note: __if no ligation site is found__, step two will be repeated using digested RE site as split point (`AAGCT` in the case of HindIII). This is done in order to be protected against sequencing errors. When this path is followed the digested RE site is removed, but not replaced. __If digested RE sites are not found either, the read will be classified as unmapped__._

_Note: __both mapping strategies can be combined__, for example defining the windows as previously (iterative mapping), but also give a RE name_ `r_enz=MboI` _and setting_ `frag_map=True` _like this if a read has not been mapped in any window, TADbit will also try to apply the fragment-based strategy._

In [2]:
%%bash 

tadbit map -w ../results/PSC_rep1 \
    --fastq ../FASTQs/PSC/PSC_HiC_rep1_subset_chr3_1.fastq.gz \
    --read 1 \
    --index ../refGenome/mm39_chr3.gem \
    --renz MboI \
    --cpus 12

Writing log to ../results/PSC_rep1/process.log
Preparing FASTQ file
  - conversion to MAP format
Mapping reads in window 1-end_e06c0841c2...
TO GEM 3 /home/3DAROC21/3DAROC21/results/PSC_rep1_tmp_r1_e06c0841c2/PSC_HiC_rep1_subset_chr3_1_c1j0cpu2
/home/3DAROC21/miniconda2/envs/tadbit/bin/gem-mapper -I /home/3DAROC21/3DAROC21/refGenome/mm39_chr3.gem -t 6 -F SAM -o /home/3DAROC21/3DAROC21/results/PSC_rep1_tmp_r1_e06c0841c2/PSC_HiC_rep1_subset_chr3_1_c1j0cpu2_full_1-end_e06c0841c2.map -i /home/3DAROC21/3DAROC21/results/PSC_rep1_tmp_r1_e06c0841c2/PSC_HiC_rep1_subset_chr3_1_c1j0cpu2
Parsing result...
   x removing GEM input /home/3DAROC21/3DAROC21/results/PSC_rep1_tmp_r1_e06c0841c2/PSC_HiC_rep1_subset_chr3_1_c1j0cpu2
   x removing map /home/3DAROC21/3DAROC21/results/PSC_rep1_tmp_r1_e06c0841c2/PSC_HiC_rep1_subset_chr3_1_c1j0cpu2_full_1-end_e06c0841c2.map
  - splitting into restriction enzyme (RE) fragments using ligation sites
  - ligation sites are replaced by RE sites to match the reference 

Writing versions of TADbit and dependencies
Generating Hi-C QC plot
  - Dangling-ends (sensu-stricto): 1.340%
  - Ligation sites: 35.204%
mapping ../FASTQs/PSC/PSC_HiC_rep1_subset_chr3_1.fastq.gz read 1 to ../results/PSC_rep1
/home/3DAROC21/miniconda2/envs/tadbit/lib/python3.7/site-packages/pytadbit/mapping/full_mapper.py:601: UserWarning: WARNING: only 145 Gb left on tmp_dir: /home/3DAROC21/3DAROC21/results/PSC_rep1_tmp_r1_e06c0841c2

  warn('WARNING: only %d Gb left on tmp_dir: %s\n' % (fspace, temp_dir))
cleaning temporary files


And for the other read-end we should change the `--fastq` file and the `--read` parameter to `2`

In [3]:
%%bash 

tadbit map -w ../results/PSC_rep1 \
    --fastq ../FASTQs/PSC/PSC_HiC_rep1_subset_chr3_2.fastq.gz \
    --read 2 \
    --index ../refGenome/mm39_chr3.gem \
    --renz MboI \
    --cpus 12

Writing log to ../results/PSC_rep1/process.log
Preparing FASTQ file
  - conversion to MAP format
Mapping reads in window 1-end_dbbd3e8e31...
TO GEM 3 /home/3DAROC21/3DAROC21/results/PSC_rep1_tmp_r2_dbbd3e8e31/PSC_HiC_rep1_subset_chr3_2_2sbavg3r
/home/3DAROC21/miniconda2/envs/tadbit/bin/gem-mapper -I /home/3DAROC21/3DAROC21/refGenome/mm39_chr3.gem -t 6 -F SAM -o /home/3DAROC21/3DAROC21/results/PSC_rep1_tmp_r2_dbbd3e8e31/PSC_HiC_rep1_subset_chr3_2_2sbavg3r_full_1-end_dbbd3e8e31.map -i /home/3DAROC21/3DAROC21/results/PSC_rep1_tmp_r2_dbbd3e8e31/PSC_HiC_rep1_subset_chr3_2_2sbavg3r
Parsing result...
   x removing GEM input /home/3DAROC21/3DAROC21/results/PSC_rep1_tmp_r2_dbbd3e8e31/PSC_HiC_rep1_subset_chr3_2_2sbavg3r
   x removing map /home/3DAROC21/3DAROC21/results/PSC_rep1_tmp_r2_dbbd3e8e31/PSC_HiC_rep1_subset_chr3_2_2sbavg3r_full_1-end_dbbd3e8e31.map
  - splitting into restriction enzyme (RE) fragments using ligation sites
  - ligation sites are replaced by RE sites to match the reference 

Writing versions of TADbit and dependencies
Generating Hi-C QC plot
  - Dangling-ends (sensu-stricto): 1.434%
  - Ligation sites: 35.818%
mapping ../FASTQs/PSC/PSC_HiC_rep1_subset_chr3_2.fastq.gz read 2 to ../results/PSC_rep1
/home/3DAROC21/miniconda2/envs/tadbit/lib/python3.7/site-packages/pytadbit/mapping/full_mapper.py:601: UserWarning: WARNING: only 143 Gb left on tmp_dir: /home/3DAROC21/3DAROC21/results/PSC_rep1_tmp_r2_dbbd3e8e31

  warn('WARNING: only %d Gb left on tmp_dir: %s\n' % (fspace, temp_dir))
cleaning temporary files


## Map files logs and format

TADbit tools uses a SQLite database file where all the commands executed by the user as well as all relevant statistics of the analysis are stored. An output of the database is shown at the end of each command.

In this case we can see useful statistics of the mapping step, for example how many reads are mapped at full length, after splitting and the percentages of dangling-ends and ligation sites:

In [4]:
%%bash

tadbit describe -w ../results/PSC_rep1 -t 4

,---------------.
| MAPPED_INPUTs |
,----.--------.------------.------.------.------.--------.---------------.-------------------.----------.-----------------.---------.
| Id | PATHid |    Entries | Trim | Frag | Read | Enzyme | Dangling_Ends |    Ligation_Sites | WRKDIRid | MAPPED_OUTPUTid | INDEXid |
|----+--------+------------+------+------+------+--------+---------------+-------------------+----------+-----------------+---------|
|  1 |      2 | 10,410,291 | None | full |    1 |   MboI |   MboI:1.340% | MboI-MboI:35.204% |        1 |               6 |       3 |
|  2 |      2 |  1,078,595 | None | frag |    1 |   MboI |   MboI:1.340% | MboI-MboI:35.204% |        1 |               7 |       3 |
|  3 |      8 | 10,410,291 | None | full |    2 |   MboI |   MboI:1.434% | MboI-MboI:35.818% |        1 |              10 |       3 |
|  4 |      8 |  1,087,028 | None | frag |    2 |   MboI |   MboI:1.434% | MboI-MboI:35.818% |        1 |              11 |       3 |
'----^--------^-----------

We can list the generated map files in the working directory in the subfolder `01_mapped_r1` and `01_mapped_r2` and inspect one of the files to see its structure:

In [5]:
%%bash

ls ../results/PSC_rep1/01_mapped_r1/
ls ../results/PSC_rep1/01_mapped_r2/

PSC_HiC_rep1_subset_chr3_1_frag_1-end_e06c0841c2.map
PSC_HiC_rep1_subset_chr3_1_full_1-end_e06c0841c2.map
PSC_HiC_rep1_subset_chr3_2_frag_1-end_dbbd3e8e31.map
PSC_HiC_rep1_subset_chr3_2_full_1-end_dbbd3e8e31.map


In [7]:
%%bash

head ../results/PSC_rep1/01_mapped_r1/PSC_HiC_rep1_subset_chr3_1_frag_1-end_e06c0841c2.map -n 2

SRR5344969.sra.180~2~	GATCATATCTACAGTCATTTTTTAAAATA	HHHHGGGGGGGGGGGGGGGGGGGGCCCCC	1	chr3:+:5690818:29
SRR5344969.sra.578	GATCCACAAAGACAAATTTGGTATGT	HHHHGGGGGGGGGGGGGGGGGCCCCC	1	chr3:-:29696661:26


A map file is a tsv file storing information of a sequence mapped to a reference genome. It is composed by the following columns:

1. Sequence identifier of the read in the FASTQ.
2. The sequence.
3. The base call quality scores, Phred +33 encoded using ASCII characters.
4. Reserved for mapper information.
5. The information of the region where the read was mapped as chromosome:strand:position:length


### Questions

- True or False? In case it is false try to correct the	sentence:
 1. The iterative mapping works only if a restriction enzyme has been used in the experiment.
 2. Iterative and fragment-based mapping are incompatible between each other.                              
 3. If a read has been mapped, the .map files contains all the information in a .fastq file for this read.
 4. If a sequence is not mapped, it doesn't appear in a .map file.                                         
- Would you be confortable to explain to your collegues the iterative mapping strategy? What about the fragment-based mapping?
- Would you be confortable to explain the difference between a read and restriction-enzyme fragment? Are they the same thing?

### References

<a name="cite-Imakaev2012a"/><sup>[^ref1](#ref-1) </sup>Imakaev, Maxim V and Fudenberg, Geoffrey and McCord, Rachel Patton and Naumova, Natalia and Goloborodko, Anton and Lajoie, Bryan R and Dekker, Job and Mirny, Leonid A. 2012. _Iterative correction of Hi-C data reveals hallmarks of chromosome organization._. [URL](http://www.ncbi.nlm.nih.gov/pubmed/22941365)

<a name="cite-Ay2015a"/><sup>[^ref2](#ref-2) </sup>Ay, Ferhat and Noble, William Stafford. 2015. _Analysis methods for studying the 3D architecture of the genome_. [URL](http://genomebiology.com/2015/16/1/183)

<a name="cite-Wingett2015"/><sup>[^ref3](#ref-3) </sup>Wingett, Steven and Ewels, Philip and Furlan-Magaril, Mayra and Nagano, Takashi and Schoenfelder, Stefan and Fraser, Peter and Andrews, Simon. 2015. _HiCUP: pipeline for mapping and processing Hi-C data._. [URL](http://f1000research.com/articles/4-1310/v1)

<a name="cite-Serra2017"/><sup>[^ref4](#ref-4) </sup>François Serra, Davide Baù, Mike Goodstadt, David Castillo,  Guillaume Filion and Marc A. Marti-Renom. 2017. _Automatic analysis and 3D-modelling of Hi-C data using TADbit reveals structural features of the fly chromatin colors_. [URL](https://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1005665)

<a name="cite-Marco-Sola2012"/><sup>[^ref5](#ref-5) </sup>Marco-Sola, Santiago and Sammeth, Michael and Guigó, Roderic and Ribeca, Paolo. 2012. _The GEM mapper: fast, accurate and versatile alignment by filtration_.

